# 1. Tratamento dos Dados

O notebook tem o objetivo de tratar os dados da seguintes bases:
1. Base de Informações Demográficas Municípios Brasileiro: base disponibilizada pela Plusoft com informações demográficas do IBGE de todos os municípios do Brasil.
2. Base do Cadastro Nacional da Pessoa Jurídica: dados do Cadastro Nacional da Pessoa Jurídica (CNPJ), com informações do registro de CNPJ por municípios.

O objetivo é agrupar a base 1 com as seguintes informação da base 2: a quantidade de CNPJs total por múnicipio, e quantidade de CNPJs com o CNAE [4711](https://concla.ibge.gov.br/busca-online-cnae.html?subclasse=4711302&tipo=cnae&versao=9&view=subclasse) relacionados a supermercados.

In [1]:
# Importando os pacotes necessários para o Notebook
import pandas as pd
import sqlite3
import numpy as np
from unidecode import unidecode

## 2. Base de Demográfica Municípios
A Plusoft disponibilizou a base de municípios com dados demográficos de todos os municípios do Brasil

In [2]:
df_municipios = pd.read_csv('../data/raw/case_selecao_2021.csv', 
                         sep = ";", 
                         decimal = ",")
df_municipios.head()

,Código,Município,Área (km²),"Densidade demográfica, 2000",Distância à capital (km),"Esperança de vida ao nascer, 2000","Mortalidade até um ano de idade, 2000","Taxa de fecundidade total, 2000","Percentual de pessoas de 25 anos ou mais analfabetas, 2000","Renda per Capita, 2000",...,"Taxa de alfabetização, 2000","Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000","População de 25 anos ou mais de idade, 1991","População de 25 anos ou mais de idade, 2000","População de 65 anos ou mais de idade, 1991","População de 65 anos ou mais de idade, 2000","População total, 1991","População total, 2000","População urbana, 2000","População rural, 2000"
0,520005,Abadia de Goiás (GO),136.9,36.3,21.414308,67.58,27.70,2.85,15.64,205.27,...,88.03,4.87,1872,2433,121,227,4227,4971,3096,1875
1,310010,Abadia dos Dourados (MG),897.4,7.2,390.876513,72.92,20.51,2.54,14.48,196.36,...,86.58,4.36,3308,3781,354,459,6492,6446,3927,2519
2,520010,Abadiânia (GO),1047.7,10.9,78.641435,69.59,22.36,2.77,21.81,168.31,...,82.45,4.24,4241,5918,446,686,9402,11452,7206,4246
3,150010,Abaetetuba (PA),1613.9,73.8,51.184552,70.77,25.61,3.15,24.85,102.14,...,80.74,4.28,33554,44897,3792,4688,99989,119152,70843,48309
4,310020,Abaeté (MG),1822.4,12.3,172.311549,72.50,21.59,2.41,15.56,253.68,...,87.55,4.78,10163,12406,1084,1595,20689,22360,19022,3338


In [3]:
df_municipios.shape

(5507, 25)

In [4]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5507 entries, 0 to 5506
Data columns (total 25 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Código                                                                 5507 non-null   int64  
 1   Município                                                              5507 non-null   object 
 2   Área (km²)                                                             5507 non-null   float64
 3   Densidade demográfica, 2000                                            5507 non-null   float64
 4   Distância à capital (km)                                               5507 non-null   float64
 5   Esperança de vida ao nascer, 2000                                      5507 non-null   float64
 6   Mortalidade até um ano de idade, 2000                                  5507 non-null   f

A base de municípios tem 5.507 registros, com 25 variáveis.

## 3. Base do Cadastro Nacional da Pessoa Jurídica
Base de dados liberada pela RFB no dia 15/05/2019 extraída do repositório: https://github.com/georgevbsantiago/qsacnpj

A base de dados está no formato de SQLite, com o seguinte schema:
<img src="../data/raw/esquema_cnpj.png">

Baseado neste schema, para extrair as informações é necessarios realizar as queries: 
* Quantidade de CNPJs total por múnicipio:

```sql 
SELECT 
    codigo_municipio, 
    municipio,
    uf,
    count(codigo_municipio) as 'QTD_TOTAL_CNPJ'
FROM cnpj_dados_cadastrais_pj
GROUP BY codigo_municipio, municipio, uf
ORDER BY QTD_TOTAL_CNPJ DESC
``` 
* Quantidade de CNPJs com o CNAE relacionados a supermercado por município: 

```sql 
SELECT 
    codigo_municipio, 
    municipio,
    uf,
    count(codigo_municipio) as 'QTD_TOTAL_CNPJ_SUPERMERCADO'
FROM cnpj_dados_cadastrais_pj
WHERE cnae_fiscal LIKE '4711%'
GROUP BY codigo_municipio, municipio, uf
ORDER BY QTD_TOTAL_CNPJ_SUPERMERCADO DESC
``` 


In [5]:
# Importando o pacote para manipulação do arquivo sqlite

conn = sqlite3.connect('../data/raw/bd_dados_qsa_cnpj.db')
cur = conn.cursor()
cur.execute('SELECT name FROM sqlite_master;')
print(cur.fetchall())

[('cnpj_dados_cadastrais_pj',), ('cnpj_dados_socios_pj',), ('cnpj_dados_cnae_secundario_pj',), ('tab_cnpj_entes_publicos',), ('tab_qualificacao_responsavel_socio',), ('tab_situacao_cadastral',), ('tab_natureza_juridica',), ('tab_cnae',), ('index_cnpj',), ('index_cnpj_socios',), ('index_cnpj_cnae',)]


In [6]:
# Contagem de Municipios dispóniveis 

In [7]:
df_total_cnpjs = pd.read_sql_query("""
            SELECT 
            codigo_municipio, 
            municipio,
            uf,
            count(codigo_municipio) as 'QTD_TOTAL_CNPJ'
            FROM cnpj_dados_cadastrais_pj
            GROUP BY codigo_municipio, municipio, uf
            ORDER BY QTD_TOTAL_CNPJ DESC
            """, conn)
df_total_cnpjs.head()

,codigo_municipio,municipio,uf,QTD_TOTAL_CNPJ
0,7107,SAO PAULO,SP,3602821
1,6001,RIO DE JANEIRO,RJ,1539550
2,4123,BELO HORIZONTE,MG,788806
3,9701,BRASILIA,DF,676800
4,7535,CURITIBA,PR,610205


In [8]:
df_cnpjs_supermercado = pd.read_sql_query("""
            SELECT 
            codigo_municipio, 
            municipio,
            uf,
            cnae_fiscal,
            count(codigo_municipio) as 'QTD_TOTAL_CNPJ_SUPERMERCADO'
            FROM cnpj_dados_cadastrais_pj
            WHERE cnae_fiscal LIKE '47113%'
            GROUP BY codigo_municipio, municipio, uf
            ORDER BY QTD_TOTAL_CNPJ_SUPERMERCADO DESC
            """, conn)
df_cnpjs_supermercado.head()

,codigo_municipio,municipio,uf,cnae_fiscal,QTD_TOTAL_CNPJ_SUPERMERCADO
0,7107,SAO PAULO,SP,4711302,22185
1,6001,RIO DE JANEIRO,RJ,4711301,7970
2,1389,FORTALEZA,CE,4711302,7435
3,0255,MANAUS,AM,4711301,5840
4,4123,BELO HORIZONTE,MG,4711301,5716


In [9]:
df_cnpjs_supermercado['cnae_fiscal'].unique()

array(['4711302', '4711301'], dtype=object)

## 4. Unificação das Bases
### 4.1. Inclusão do ajuste dos campos Munícipio e UF da Base Demográfica

Para realizar a junção das bases Demográfica com a base de CNPJs devemos realizar um ajuste nas coluna de muncipio e UF, retirando acentos e colocando o texto maiúsculo. 


In [10]:
df_municipios['uf_ajustado'] = df_municipios['Município'] \
                                            .str.slice(start = -3) \
                                            .str.replace(")", "", regex = True) \
                                            .str.strip()

In [11]:
df_municipios['uf_ajustado'].unique()

array(['GO', 'MG', 'PA', 'CE', 'BA', 'PR', 'SC', 'PE', 'TO', 'RN', 'PI',
       'MT', 'AC', 'MA', 'SP', 'ES', 'RS', 'PB', 'MS', 'RO', 'RR', 'AM',
       'AP', 'SE', 'AL', 'RJ', 'DF'], dtype=object)

In [12]:
df_municipios['municipio_ajustado'] = df_municipios['Município'] \
                                            .str.split('(', expand = True)[0] \
                                            .str.strip()
for i in range(0, len(df_municipios['municipio_ajustado'])):
    municipio = df_municipios['municipio_ajustado'][i]
    municipio = unidecode(municipio.upper())
    df_municipios['municipio_ajustado'][i] = municipio

    
df_municipios['municipio_ajustado']

<ipython-input-12-ebf3628eb597>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_municipios['municipio_ajustado'][i] = municipio


0           ABADIA DE GOIAS
1       ABADIA DOS DOURADOS
2                 ABADIANIA
3                ABAETETUBA
4                    ABAETE
               ...         
5502            XIQUE-XIQUE
5503                 ZABELE
5504               ZACARIAS
5505                ZE DOCA
5506                 ZORTEA
Name: municipio_ajustado, Length: 5507, dtype: object

In [13]:
df_municipios[['municipio_ajustado', 'uf_ajustado']].head()

,municipio_ajustado,uf_ajustado
0,ABADIA DE GOIAS,GO
1,ABADIA DOS DOURADOS,MG
2,ABADIANIA,GO
3,ABAETETUBA,PA
4,ABAETE,MG


In [14]:
# Ajuste das Localidades com o Nome diferente
df_municipios['municipio_ajustado'].replace("MOJI DAS CRUZES", "MOGI DAS CRUZES", inplace = True)
df_municipios.iloc[1635:1636]['municipio_ajustado'].replace("EMBU", "EMBU DAS ARTES", inplace = True)
municipios = ["MOGI DAS CRUZES", "EMBU DAS ARTES"]
df_municipios.query('municipio_ajustado in @municipios')

/home/danielvolponi/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,Código,Município,Área (km²),"Densidade demográfica, 2000",Distância à capital (km),"Esperança de vida ao nascer, 2000","Mortalidade até um ano de idade, 2000","Taxa de fecundidade total, 2000","Percentual de pessoas de 25 anos ou mais analfabetas, 2000","Renda per Capita, 2000",...,"População de 25 anos ou mais de idade, 1991","População de 25 anos ou mais de idade, 2000","População de 65 anos ou mais de idade, 1991","População de 65 anos ou mais de idade, 2000","População total, 1991","População total, 2000","População urbana, 2000","População rural, 2000",uf_ajustado,municipio_ajustado
1635,351500,Embu (SP),70.3,2941.4,24.692624,69.99,18.26,2.44,9.37,244.20,...,65777,97757,2971,5024,155990,207663,207663,0,SP,EMBU DAS ARTES
3074,353060,Moji das Cruzes (SP),727.1,453.4,45.725004,68.50,21.42,2.15,7.75,386.11,...,126323,171260,11117,16998,263336,330241,302116,28125,SP,MOGI DAS CRUZES


### 4.2. Unificação das Bases

In [15]:
# Merge das bases do municípios e base total de CNPJs
df_unificada = df_municipios.merge(df_total_cnpjs[['municipio', 'uf', 'QTD_TOTAL_CNPJ']], 
                    how = 'left',
                   left_on = ['municipio_ajustado', 'uf_ajustado'],
                   right_on = ['municipio', 'uf'],
                   suffixes=('rf_','right')).drop(columns = ['municipio', 'uf'])
df_unificada.head()

,Código,Município,Área (km²),"Densidade demográfica, 2000",Distância à capital (km),"Esperança de vida ao nascer, 2000","Mortalidade até um ano de idade, 2000","Taxa de fecundidade total, 2000","Percentual de pessoas de 25 anos ou mais analfabetas, 2000","Renda per Capita, 2000",...,"População de 25 anos ou mais de idade, 2000","População de 65 anos ou mais de idade, 1991","População de 65 anos ou mais de idade, 2000","População total, 1991","População total, 2000","População urbana, 2000","População rural, 2000",uf_ajustado,municipio_ajustado,QTD_TOTAL_CNPJ
0,520005,Abadia de Goiás (GO),136.9,36.3,21.414308,67.58,27.70,2.85,15.64,205.27,...,2433,121,227,4227,4971,3096,1875,GO,ABADIA DE GOIAS,2216.0
1,310010,Abadia dos Dourados (MG),897.4,7.2,390.876513,72.92,20.51,2.54,14.48,196.36,...,3781,354,459,6492,6446,3927,2519,MG,ABADIA DOS DOURADOS,1131.0
2,520010,Abadiânia (GO),1047.7,10.9,78.641435,69.59,22.36,2.77,21.81,168.31,...,5918,446,686,9402,11452,7206,4246,GO,ABADIANIA,2550.0
3,150010,Abaetetuba (PA),1613.9,73.8,51.184552,70.77,25.61,3.15,24.85,102.14,...,44897,3792,4688,99989,119152,70843,48309,PA,ABAETETUBA,10264.0
4,310020,Abaeté (MG),1822.4,12.3,172.311549,72.50,21.59,2.41,15.56,253.68,...,12406,1084,1595,20689,22360,19022,3338,MG,ABAETE,5588.0


In [16]:
# Merge das bases do municípios e base de CNPJs por supermercado
df_unificada = df_unificada.merge(df_cnpjs_supermercado[['municipio', 'uf', 'QTD_TOTAL_CNPJ_SUPERMERCADO']], 
                   how = 'left',
                   left_on = ['municipio_ajustado', 'uf_ajustado'],
                   right_on = ['municipio', 'uf'],
                   suffixes=('rf_','right')).drop(columns = ['municipio', 'uf', 'municipio_ajustado', 'uf_ajustado'])
df_unificada.head()

,Código,Município,Área (km²),"Densidade demográfica, 2000",Distância à capital (km),"Esperança de vida ao nascer, 2000","Mortalidade até um ano de idade, 2000","Taxa de fecundidade total, 2000","Percentual de pessoas de 25 anos ou mais analfabetas, 2000","Renda per Capita, 2000",...,"População de 25 anos ou mais de idade, 1991","População de 25 anos ou mais de idade, 2000","População de 65 anos ou mais de idade, 1991","População de 65 anos ou mais de idade, 2000","População total, 1991","População total, 2000","População urbana, 2000","População rural, 2000",QTD_TOTAL_CNPJ,QTD_TOTAL_CNPJ_SUPERMERCADO
0,520005,Abadia de Goiás (GO),136.9,36.3,21.414308,67.58,27.70,2.85,15.64,205.27,...,1872,2433,121,227,4227,4971,3096,1875,2216.0,22.0
1,310010,Abadia dos Dourados (MG),897.4,7.2,390.876513,72.92,20.51,2.54,14.48,196.36,...,3308,3781,354,459,6492,6446,3927,2519,1131.0,23.0
2,520010,Abadiânia (GO),1047.7,10.9,78.641435,69.59,22.36,2.77,21.81,168.31,...,4241,5918,446,686,9402,11452,7206,4246,2550.0,36.0
3,150010,Abaetetuba (PA),1613.9,73.8,51.184552,70.77,25.61,3.15,24.85,102.14,...,33554,44897,3792,4688,99989,119152,70843,48309,10264.0,254.0
4,310020,Abaeté (MG),1822.4,12.3,172.311549,72.50,21.59,2.41,15.56,253.68,...,10163,12406,1084,1595,20689,22360,19022,3338,5588.0,62.0


### 4.3. Verificando os valores não encontrados

In [17]:
df_unificada[['Município', 'População total, 2000', 'QTD_TOTAL_CNPJ_SUPERMERCADO']] \
                                .query('QTD_TOTAL_CNPJ_SUPERMERCADO != QTD_TOTAL_CNPJ_SUPERMERCADO') \
                                .sort_values(by = 'População total, 2000', ascending = False)

,Município,"População total, 2000",QTD_TOTAL_CNPJ_SUPERMERCADO
3070,Mogi Guaçu (SP),124228,NaN
3073,Moji-Mirim (SP),81467,NaN
2633,lagoas (MG),63605,NaN
27,Açu (RN),47904,NaN
2327,Itapagé (CE),41093,NaN
...,...,...,...
3943,Pracinha (SP),1431,NaN
1417,Coxixola (PB),1422,NaN
5322,União Paulista (SP),1354,NaN
4100,Riacho de Santo Antônio (PB),1334,NaN


Removendo os valores NaN

In [18]:
df_unificada = df_unificada.dropna()

## 5. Exportando a base Unificada
Exportando a base para o notebook para Análise Exploratória de Dados

In [19]:
df_unificada.to_csv('../data/interim/base_unificada.csv',
                   sep=";",
                   index=False)